In [3]:
from nltk.corpus import treebank, ptb
from collections import Counter
import nltk
import pickle as pkl

## Data loading and analysis

In [49]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/rj1408/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [7]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /home/rj1408/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [6]:
nltk.download('ptb')

[nltk_data] Downloading package ptb to /home/rj1408/nltk_data...
[nltk_data]   Unzipping corpora/ptb.zip.


True

In [17]:
#total number of words/sents in corpus
len(ptb.tagged_sents()), len(ptb.tagged_words())

(49208, 1253013)

In [26]:
allfileids = ptb.fileids()
allfileids = list(map(lambda fid: (fid, fid.split('/')[1]), allfileids))
trainfileids = list(filter(lambda tup: int(tup[1]) <= 18, allfileids))
valfileids = list(filter(lambda tup: int(tup[1]) > 18 and int(tup[1]) <= 21, allfileids))
testfileids = list(filter(lambda tup: int(tup[1]) >= 22, allfileids))
trainfileids = list(map(lambda tup: tup[0], trainfileids))
valfileids = list(map(lambda tup: tup[0], valfileids))
testfileids = list(map(lambda tup: tup[0], testfileids))

In [54]:
traindict = {}
traindict['tagged_words'] = list(ptb.tagged_words(fileids=trainfileids))
traindict['tagged_sents'] = list(ptb.tagged_sents(fileids=trainfileids))
valdict = {}
valdict['tagged_words'] = list(ptb.tagged_words(fileids=valfileids))
valdict['tagged_sents'] = list(ptb.tagged_sents(fileids=valfileids))
testdict = {}
testdict['tagged_words'] = list(ptb.tagged_words(fileids=testfileids))
testdict['tagged_sents'] = list(ptb.tagged_sents(fileids=testfileids))

In [56]:
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/train.p","wb") as f:
    pkl.dump( traindict, f)
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/val.p","wb") as f:
    pkl.dump( valdict, f)
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/test.p","wb") as f:
    pkl.dump( testdict, f)

In [4]:
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/train.p","rb") as f:
    traindict = pkl.load(f)
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/val.p","rb") as f:
    valdict = pkl.load(f)
with open("/scratch/rj1408/pos_lm/ptb_wsj_pos/test.p","rb") as f:
    testdict = pkl.load(f)

In [5]:
#tag analysis
with open('tagset.txt') as f:
    alltags = f.read()

alltags = list(map(lambda strline: strline.split('\t')[1], alltags.split('\n')))
alltags = set(alltags)

In [59]:
#get distribution of tags
tagcntr = Counter(map(lambda tupl: tupl[1], filter(lambda tup: tup[1] in alltags, traindict['tagged_words'])))
tagcntr.most_common()

[('NN', 127563),
 ('IN', 94760),
 ('NNP', 87693),
 ('DT', 78777),
 ('JJ', 58957),
 ('NNS', 57860),
 ('CD', 34891),
 ('RB', 29621),
 ('VBD', 28311),
 ('VB', 25489),
 ('CC', 22832),
 ('TO', 21462),
 ('VBZ', 20982),
 ('VBN', 19333),
 ('PRP', 16766),
 ('VBG', 14350),
 ('VBP', 12326),
 ('MD', 9437),
 ('POS', 8284),
 ('PRP$', 7989),
 ('WDT', 4194),
 ('JJR', 3174),
 ('RP', 2515),
 ('NNPS', 2505),
 ('WP', 2285),
 ('WRB', 2051),
 ('JJS', 1867),
 ('RBR', 1675),
 ('EX', 833),
 ('RBS', 435),
 ('PDT', 333),
 ('FW', 224),
 ('WP$', 166),
 ('UH', 87),
 ('SYM', 55),
 ('LS', 47)]

## Base model - Majority class voting

In [11]:
def getTokenAccuracy(labels, preds):
    alllabels = [tok for lis in labels for tok in lis]
    allpreds= [tok for lis in preds for tok in lis]
    cnt = 0
    for i,tok in enumerate(alllabels):
        if alllabels[i]==allpreds[i]:
            cnt += 1
    return float(cnt)/len(alllabels)    

def getSentAccuracy(labels, preds):
    cnt = 0
    for i,sent in enumerate(labels):
        if labels[i]==preds[i]:
            cnt += 1
    return float(cnt)/len(labels)

In [7]:
def pruneNonLabels(labels, preds, alltags):
    useless = list(map(lambda lis: list(filter(lambda tup: tup[1] not in alltags, list(enumerate(lis)))), labels))
    useless = list(map(lambda lis: set(map(lambda tup: tup[0], lis)), useless))
    prunedLabels = list(map(lambda lis: list(filter(lambda lab: lab in alltags, lis)), labels))
    prunedPreds = list(map(lambda tup: list(filter(lambda tupl: tupl[0] not in useless[tup[0]], list(enumerate(tup[1])))), list(enumerate(preds))))
    prunedPreds = list(map(lambda lis: list(map(lambda tupl: tupl[1], lis)), prunedPreds))
    itemsLabels = [tok for lis in prunedLabels for tok in lis]
    itemsPruned = [tok for lis in prunedPreds for tok in lis]
    assert len(itemsLabels) == len(itemsPruned)
    return prunedLabels, prunedPreds
    
def getPredictions(lisoflisof_tokens, wrd_cntr):
    return list(map(lambda lisoftokens: list(map(lambda tok: "-1" if tok not in wrd_cntr else wrd_cntr[tok].most_common(1)[0][0], lisoftokens)), lisoflisof_tokens))

In [8]:
lis = traindict['tagged_words']
wrd_cntr = {}
for tup in lis:
    if tup[0] not in wrd_cntr:
        wrd_cntr[tup[0]] = Counter()
    wrd_cntr[tup[0]][tup[1]] += 1

In [9]:
test_sents = list(map(lambda lis: list(map(lambda tup: tup[0], lis)), testdict['tagged_sents']))
predictions = getPredictions(test_sents, wrd_cntr)
labels = list(map(lambda lis: list(map(lambda tup: tup[1], lis)), testdict['tagged_sents']))
labels, predictions = pruneNonLabels(labels, predictions, alltags)

In [14]:
getTokenAccuracy(labels, predictions), getSentAccuracy(labels, predictions)

(0.90011926321834, 0.169718051995606)